In [1]:
# -----------------------------------------------------------
# На позицию
# Стажер - АО «Центр развития платежных и финансовых технологий Национального Банка РК»

# Первая задача:  
# Парсинг сайта kingfisher.kz
# Условие задания
#  Записать в базу данных:
#    Название продукта
#    Название категории продукта
#    Цену продукта
#    Название города
# 
# Результат отправить ссылкой на github  
# -----------------------------------------------------------

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import sqlite3 

In [3]:
url = 'https://kingfisher.kz/'
r = requests.get(url) 
soup = BeautifulSoup(r.text)

uls_lis = soup.find_all('li',{'class': 'dropmenu'})
soup_uld = BeautifulSoup(str(uls_lis)) 

dropmenu = soup_uld.find_all('li', {'class': 'dropmenu'}) # категория ишиндеги тамак аттары

In [4]:
good_soup = BeautifulSoup(str(dropmenu)) 
good_soup_1 = good_soup.find_all("span")
category = [str(i).replace("<span>", '').replace("</span>", '') for i in good_soup_1]
category = [category[i] for i in range(len(category)) if str(category[i])!= "Назад" ]

In [5]:
city_span_c = str(BeautifulSoup(str(soup.find_all('div',{'class': 'popupBtn_city'}))).find_all("span")).replace('[<span>', '').replace("</span>]", '')

In [6]:
def get_one_c(i):
    soup_temp = BeautifulSoup(str(dropmenu[i])) 

    category_temp = soup_temp.find_all("span")

    category_temp_new = str(category_temp).replace("<span>", '').replace("</span>", '')

    if (category_temp_new.find(category[i]) == 1):
        return(category[i])
# to get one category

In [7]:
def get_links(i):
    soup_soup = BeautifulSoup(str(dropmenu[i])) 
    links = [] # link
    for link in soup_soup.find_all('a', href=True):
        links.append(link['href'])
    links_1 = []
    # if / > 3 delete
    len_1 = []
    c = 0
    for i in links:
        for j in i:
            if(j == '/'):
                c = 1 + c
        len_1.append(c)
        c = 0
    
    for j in range(len(links)):
        if(len_1[j] == 3):
            links_1.append(links[j])
    return(links_1)

In [8]:
# Создание датафрэйма
df = pd.DataFrame(columns = ['product_id','product_category_name','product_name','product_price','product_city'])
df_1 =  pd.DataFrame(columns = ['product_category_name','product_link'])

In [9]:
# Чтобы получуть отдельно 'product_category_name','product_link'
count = 0 
for i in range(len(dropmenu)):
    for j in get_links(i):
        df_1.loc[count] = ([str(category[i]), str(j)])
        count = count + 1

In [10]:
def get_links_names_ins(i):
    soup_soup = BeautifulSoup(str(goodsBlock[i])) 
    links_n = [] # text
    for link_n in soup_soup.find_all('a'):
        links_n.append(link_n.getText())
    return(links_n[1])

In [11]:
def get_price_ins(i):
    price_span_pos = goodsBlock[i].find_all('span', {'class': 'pricePosition'})
    soup_price = BeautifulSoup(str(price_span_pos)) 
    price_span_real = soup_price.find_all('span', {'class': 'new'})
    price_new = str(price_span_real).replace('[<span class="new">', '').replace("Т</span>]", '').replace(' ', '')
    price = int(price_new)
    return(price)

In [12]:
# Выполнение парсинга и заточение данных в датафрейм
count_4 = 0
for i in range(len(df_1)):
    for j in range(len(category)):
        if(df_1['product_category_name'][i] == category[j]):
            url_ins = 'https://kingfisher.kz' + df_1['product_link'][i]
            r_ins = requests.get(url_ins) 
            soup_ins = BeautifulSoup(r_ins.text)
            uls_ins = soup_ins.find_all('div',{'class': 'Goods goodslist padSpace'})
            soup_n_ins = BeautifulSoup(str(uls_ins)) 
            goodsBlock = soup_n_ins.find_all('div', {'class': 'goodsBlock'})
            for k in range(len(goodsBlock)):
                df.loc[count_4] = [ count_4, str(df_1['product_category_name'][i]),str(get_links_names_ins(k)), get_price_ins(k), str(city_span_c)]
                count_4 = count_4 + 1

In [13]:
pip install openpyxl # in order to work with excel

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [14]:
# Из датафрэйма в excel
df.to_excel("pandas_df_excel.xlsx")  

In [15]:
# Из датафрэйма в базу данных sqlite3
conn = sqlite3.connect('kingsite.db')
cur = conn.cursor()

In [16]:
df.to_sql('king', con=conn,if_exists = 'replace', index=False)
# if_exists = 'replace' -- Если такая таблица существует то drop the table

In [17]:
pd.read_sql('SELECT * FROM king', conn) # show table from sql

,product_id,product_category_name,product_name,product_price,product_city
0,0,Морепродукты,Креветки Королевские 26/30 очищенные,5290,Алматы
1,1,Морепродукты,Креветка Королевская 31/40 очищенная,5900,Алматы
2,2,Морепродукты,"Лангустины в панцире с головой, размер 21-30",6500,Алматы
3,3,Морепродукты,"Креветка Королевская 21/25, в панцире без головы",5500,Алматы
4,4,Морепродукты,Чука салат замороженный,2800,Алматы
...,...,...,...,...,...
342,342,Масло,Масло Авокадо Avocado Oil,4000,Алматы
343,343,Масло,Масло Сливочное BURRO ASOLO,1790,Алматы
344,344,Кухня,Сковорода для Паэльи 38 см,9000,Алматы
345,345,Кухня,Чугунная Сковорода Гриль VAELLO,12500,Алматы


In [18]:
conn.close()